In [3]:
import tarantool
connection = tarantool.connect(host="localhost", port=3302, user='admin', password='pass') # 


In [21]:
'''Команды в консоли тарантула (запущенного в докере)
box.schema.space.create('tester2')

box.space.tester2:format({
    { name = 'id', type = 'unsigned' },
    { name = 'band_name', type = 'string' },
    { name = 'year', type = 'unsigned' }
})

-- Create a primary index --
box.space.tester2:create_index('primary', { parts = { 'id' } })

box.space.tester2:create_index('name', { parts = { 'band_name' } })
'''
space_name = 'tester2'

In [16]:
conn_space = connection.space(space_name)
res = conn_space.select()
res

- [1, 'ABBA', 1972]

In [15]:
res = connection.select(space_name, 0)
res

In [14]:
conn_space.insert((1, 'ABBA', 1972))

- [1, 'ABBA', 1972]

In [13]:
conn_space

In [24]:
res = conn_space.select('ABBA', index=1) # if index 1 already created
res

- [1, 'ABBA', 1972]

In [23]:
def select_by_name(name='ABBA'):
    res = conn_space.select(name, index='name')
    return res

print(select_by_name())

- [1, 'ABBA', 1972]


In [25]:
conn_space.upsert((1, 'ABBA', 0), [('=', 2, 16)]) # 2 - index from 0
res = select_by_name()
res

- [1, 'ABBA', 16]

In [26]:
conn_space.update(1, [('=', 1, 'ABBA'), ('+', 2, 1)]) # updated 2 fields

- [1, 'ABBA', 17]

In [27]:
# Теперь с помощью функции replace мы полностью заменим кортеж 
# с совпадающим первичным ключом. Если кортежа с указанным первичным ключом не существует, 
# то эта операция ни к чему не приведет.

conn_space.replace((4, 'New band', 2015))
res = select_by_name('New band')
res


- [4, 'New band', 2015]